In [24]:
from numba import njit
import numpy as np

In [25]:
from hftbacktest import BacktestAsset, HashMapMarketDepthBacktest

asset = (
    BacktestAsset()
        # Sets the data to feed for this asset.
        #
        # Due to the vast size of tick-by-tick market depth and trade data,
        # loading the entire dataset into memory can be challenging,
        # particularly when backtesting across multiple days.
        # HftBacktest offers lazy loading support and is compatible with npy and preferably npz.
        #
        # For details on the normalized feed data, refer to the following documents.
        # * https://hftbacktest.readthedocs.io/en/latest/data.html
        # * https://hftbacktest.readthedocs.io/en/latest/tutorials/Data%20Preparation.html
        .data(['data\\binance_future\solusdt_20251011.npz'])
        # Sets the initial snapshot (optional).
        .initial_snapshot('data\\binance_future\solusdt_20251010_eod.npz')
        # Asset type:
        # * Linear
        # * Inverse.
        # 1.0 represents the contract size, which is the value of the asset per quoted price.
        .linear_asset(1.0)
        # HftBacktest provides two built-in latency models.
        # * constant_latency
        # * intp_order_latency
        # To implement your own latency model, please use Rust.
        #
        # Time unit is the same as data's timestamp's unit. Timestamp of the sample data is in nanoseconds.
        # Sets the order entry latency and response latency to 10ms.
        .constant_latency(10_000_000, 10_000_000)
        # HftBacktest provides several types of built-in queue position models.
        # Please find the details in the documents below.
        # https://hftbacktest.readthedocs.io/en/latest/tutorials/Probability%20Queue%20Models.html
        #
        # To implement your own queue position model, please use Rust.
        .risk_adverse_queue_model()
        # HftBacktest provides two built-in exchange models.
        # * no_partial_fill_exchange
        # * partial_fill_exchange
        # To implement your own exchange model, please use Rust.
        .no_partial_fill_exchange()
        # HftBacktest provides several built-in fee models.
        # * trading_value_fee_model
        # * trading_qty_fee_model
        # * flat_per_trade_fee_model
        #
        # 0.02% maker fee and 0.07% taker fee. If the fee is negative, it represents a rebate.
        # For example, -0.00005 represents a 0.005% rebate for the maker order.
        .trading_value_fee_model(0.0001, 0.0003)
        # Tick size of this asset: minimum price increasement
        .tick_size(0.01)
        # Lot size of this asset: minimum trading unit.
        .lot_size(0.01)
        # Sets the capacity of the vector that stores trades occurring in the market.
        # If you set the size, you need call `clear_last_trades` to clear the vector.
        # A value of 0 indicates that no market trades are stored. (Default)
        .last_trades_capacity(0)
)



In [ ]:
from numba import njit

@njit
def print_3depth(hbt):
    while hbt.elapse(60 * 1e9) == 0:
        print('current_timestamp:', hbt.current_timestamp)

        # Gets the market depth for the first asset, in the same order as when you created the backtest.
        depth = hbt.depth(0)

        # a key of bid_depth or ask_depth is price in ticks.
        # (integer) price_tick = rice / tick_size
        i = 0
        for price_tick in range(depth.best_ask_tick, depth.best_ask_tick + 100):
            qty = depth.ask_qty_at_tick(price_tick)
            if qty > 0:
                print(
                    'ask: ',
                    qty,
                    '@',
                    np.round(price_tick * depth.tick_size, 2)
                )

                i += 1
                if i == 3:
                    break
        i = 0
        for price_tick in range(depth.best_bid_tick, max(depth.best_bid_tick - 100, 0), -1):
            qty = depth.bid_qty_at_tick(price_tick)
            if qty > 0:
                print(
                    'bid: ',
                    qty,
                    '@',
                    np.round(price_tick * depth.tick_size, 2)
                )

                i += 1
                if i == 3:
                    break
    return True

In [27]:
import numpy as np

solusdt_20240809 = np.load('data\\binance_future\solusdt_20251011.npz')['data']
btcusdt_20240808_eod = np.load('data\\binance_future\solusdt_20251010_eod.npz')['data']

In [28]:
from hftbacktest import BacktestAsset, HashMapMarketDepthBacktest

asset = (
    BacktestAsset()
        .data(solusdt_20240809)
        .initial_snapshot(btcusdt_20240808_eod)
        .linear_asset(1.0)
        .constant_latency(int(0.01 * 1e9), int(0.01 * 1e9))
        .risk_adverse_queue_model()
        .no_partial_fill_exchange()
        .trading_value_fee_model(0.0001, 0.0003)
        .tick_size(0.01)
        .lot_size(0.01)
)

hbt = HashMapMarketDepthBacktest([asset])

print_3depth(hbt)

_ = hbt.close()

current_timestamp: 1760140860011000000
ask:  13.23 @ 187.58
ask:  4.26 @ 187.57
ask:  1164.62 @ 187.56
bid:  26.9 @ 186.57
bid:  171.09 @ 186.56
bid:  48.11 @ 186.55
current_timestamp: 1760140920011000000
ask:  20.88 @ 187.91
ask:  44.98 @ 187.9
ask:  0.62 @ 187.89
bid:  67.3 @ 186.9
bid:  49.04 @ 186.89
bid:  78.39 @ 186.88
current_timestamp: 1760140980011000000
ask:  45.67 @ 187.74
ask:  296.55 @ 187.73
ask:  41.5 @ 187.72
bid:  347.67 @ 186.73
bid:  80.85 @ 186.72
bid:  42.74 @ 186.71
current_timestamp: 1760141040011000000
ask:  5.62 @ 189.16
ask:  0.18 @ 189.15
ask:  1.86 @ 189.14
bid:  58.78 @ 188.15
bid:  451.73 @ 188.14
bid:  22.16 @ 188.13
current_timestamp: 1760141100011000000
ask:  442.74 @ 189.82
ask:  22.29 @ 189.81
ask:  1502.11 @ 189.8
bid:  223.02 @ 188.81
bid:  76.65 @ 188.8
bid:  82.12 @ 188.79
current_timestamp: 1760141160011000000
ask:  184.04 @ 189.65
ask:  25.51 @ 189.64
ask:  5.9 @ 189.63
bid:  16.53 @ 188.64
bid:  93.08 @ 188.63
bid:  13.58 @ 188.62
current_times